# Regression Week 5: LASSO (coordinate descent)

In this notebook, you will implement your very own LASSO solver via coordinate descent. You will:
* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

# Fire up Turi Create

In [1]:
import turicreate

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = turicreate.SFrame('./data/home_data.sframe/')
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float and then to int, before using it below
sales['floors'] = sales['floors'].astype(float).astype(int)

If we want to do any "feature engineering" like creating new features or adjusting existing ones we should do this directly using the SFrames as seen in the first notebook of Week 2. For this notebook, however, we will work with the existing features.

# Import useful functions from previous notebook

As in Week 2, we convert the SFrame into a 2D Numpy array. 

In [3]:
import numpy as np # note this allows us to refer to numpy as np instead 

In [4]:
import sys
sys.path.append("../common")

from utils import get_numpy_data, predict_output, calc_rss

# Normalize features
In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features** as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [5]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
X

array([[ 3.,  5.,  8.],
       [ 4., 12., 15.]])

Numpy provides a shorthand for computing 2-norms of each column:

In [6]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
norms

array([ 5., 13., 17.])

To normalize, apply element-wise division:

In [7]:
X / norms # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

array([[0.6       , 0.38461538, 0.47058824],
       [0.8       , 0.92307692, 0.88235294]])

Using the shorthand we just covered, write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. <br />
The function should return a pair `(normalized_features, norms)`, where the second item contains the norms of original features. <br /> 
As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data. 

In [8]:
def normalize_features(f_matrix):
  norms = np.linalg.norm(f_matrix, axis=0)
  return (f_matrix / norms, norms)

To test the function, run the following:

In [9]:
features, norms = normalize_features(np.array([[3., 6., 9.], [4., 8., 12.]]))

np.testing.assert_array_equal(
  features, 
  np.array([[0.6,  0.6,  0.6], [0.8,  0.8, 0.8]])
)

# assert norms == np.array([5.,  10.,  15.])
np.testing.assert_array_equal(
  norms,
  np.array([5.,  10.,  15.])
)

# Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
$$
\Sigma (prediction - output)^2 + \lambda \times \Sigma_{j=1}^{k} | w_j | 
$$ 

(By convention, we do not include $w_0$ in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
$$
    argmin_{w_i}[\Sigma (prediction - output)^2 + \lambda \times \Sigma_{j=1}^{k} | w_j |]
$$

where all weights other than $w_i$ are held to be constant. We will optimize one $w_i$ at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute $w_i$ that minimizes the cost function $\Sigma[ (prediction - output)^2 ] + \lambda \times \Sigma_{j=1}^{k} | w_j |$
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
$$
w_i = \left\{
    \begin{array}{lr}
      \rho_i + \frac{\lambda}{2} & if \rho_i < -\frac{\lambda}{2} \\
       0  &                        if -\frac{\lambda}{2} \le \rho_i \le \frac{\lambda}{2} \\
      \rho_i - \frac{\lambda}{2} & if \rho_i > \frac{\lambda}{2} \\
    \end{array}
\right.
$$
where $\rho_i = \Sigma [ [feature_i] \times (output - prediction + w\_i \times [feature_i]) ]$

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
$w_0 = \rho_i$


## Effect of L1 penalty

Let us consider a simple model with 2 features:

In [44]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
simple_feature_matrix

array([[1.00e+00, 1.18e+03, 3.00e+00],
       [1.00e+00, 2.57e+03, 3.00e+00],
       [1.00e+00, 7.70e+02, 2.00e+00],
       ...,
       [1.00e+00, 1.02e+03, 2.00e+00],
       [1.00e+00, 1.60e+03, 3.00e+00],
       [1.00e+00, 1.02e+03, 2.00e+00]])

Don't forget to normalize features:

In [45]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)
simple_feature_matrix

array([[0.00680209, 0.00353021, 0.00583571],
       [0.00680209, 0.00768869, 0.00583571],
       [0.00680209, 0.00230361, 0.00389048],
       ...,
       [0.00680209, 0.00305154, 0.00389048],
       [0.00680209, 0.00478673, 0.00583571],
       [0.00680209, 0.00305154, 0.00389048]])

We assign some random set of initial weights and inspect the values of `ro[i]`:

In [12]:
weights = np.array([1., 4., 1.])

Use `predict_output()` to make predictions on this data.

In [13]:
prediction = predict_output(simple_feature_matrix, weights)
prediction, prediction.shape

(array([0.02675867, 0.04339256, 0.01990703, ..., 0.02289873, 0.03178473,
        0.02289873]),
 (21613,))

Compute the values of `ro[i]` for each feature in this simple model, using the formula given above, using the formula:

$\rho_i = \Sigma [ [feature_i] \times (output - prediction + w\_i \times [feature_i]) ]$

*Hint: You can get a Numpy vector for feature_i using:*
```
simple_feature_matrix[:,i]
```

In [14]:
ro = [
  np.sum(simple_feature_matrix[:, i] * (output - prediction + weights[i] * simple_feature_matrix[:, i])) for i in range(len(weights))  
]
ro

[79400300.03492916, 87939470.77299108, 80966698.67596565]

***Quiz Question***

Recall that, whenever `ro[i]` falls between `-l1_penalty/2` and `l1_penalty/2`, the corresponding weight `w[i]` is sent to zero. <br /> 
Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. <br />  
What range of values of `l1_penalty` **would not** set `w[1]` to zero, but **would** set `w[2]` to zero, if we were to take a step in that coordinate? 


by definition: 
  - for w[2]: $|\rho_2| \le \frac{l_1}{2} \equiv  l_1 \ge 2 * \rho_2$ 
  - for w[1]: $|\rho_1| \le \frac{l_1}{2} \equiv  l_1 \ge 2 * \rho_1$ 

In [15]:
# l1 range that would set w[2] to zero (but not w[1])
min(ro[2] * 2, ro[1] * 2)

161933397.3519313

***Quiz Question***

What range of values of `l1_penalty` would set **both** `w[1]` and `w[2]` to zero, if we were to take a step in that coordinate? 

In [16]:
## Any value strictly greater than
max(ro[2] * 2, ro[1] * 2)

175878941.54598215

So we can say that `ro[i]` quantifies the significance of the i-th feature: the larger `ro[i]` is, the more likely it is for the i-th feature to be retained.

## Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [17]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    prediction = predict_output(feature_matrix, weights)  # compute prediction
    
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = np.sum(feature_matrix[:, i] * (output - prediction + weights[i] * feature_matrix[:, i])) 

    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i
        
    elif ro_i < -l1_penalty / 2.:
        new_weight_i = ro_i + l1_penalty / 2.
        
    elif ro_i > l1_penalty / 2.:
        new_weight_i = ro_i - l1_penalty / 2.
        
    else:
        new_weight_i = 0.
    
    return new_weight_i

To test the function, run the following cell:

In [18]:
# should print 0.425558846691
import math

EPS = 1e-7
exp = 0.425558846691
act = lasso_coordinate_descent_step(
  1, 
  np.array([[3. / math.sqrt(13), 1. / math.sqrt(10)], [2. / math.sqrt(13), 3. / math.sqrt(10)]]), 
  np.array([1., 1.]), 
  np.array([1., 4.]), 
  0.1)

assert abs(act - exp) <= EPS, f"Expected {exp}, got: {act}"

## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**
```python
for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    ...
```

In [19]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
  weights = initial_weights
  
  while True:
    coord = []
    for i in range(len(weights)):
      old_weights_i = weights[i]
      weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
      coord.append(abs(old_weights_i - weights[i]))
      
    #
    if max(coord) < tolerance:
      break
  #
  return weights    

In [20]:
## utility functions, for the coming questions...

def features_nzw(features, weights):
  """features with non-zero weigths"""
  return list(
    filter(lambda t: t[1] > 0., 
           zip(['intercept', *features], weights))
  )

def features_zw(features, weights, eps=1e-7):
  """features with zero weigths"""
  return list(
    filter(lambda t: abs(t[1]) <= eps, 
           zip(['intercept', *features], weights))  # intercept is constant in the quiz
  )

Using the following parameters, learn the weights on the sales dataset. 

In [21]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`.

In [22]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

Then, run your implementation of LASSO coordinate descent:

In [23]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

weights, weights.shape

(array([21624998.36636292, 63157246.78545421,        0.        ]), (3,))

***Quiz Questions***
1. What is the RSS of the learned model on the normalized dataset? (Hint: use the normalized feature matrix when you make predictions.)
2. Which features had weight zero at convergence?

In [24]:
## 1. RSS
rss = calc_rss(normalized_simple_feature_matrix, output, weights)
print(f"rss: {rss} / in scientifc notation: {rss:1.5e}")

rss: 1630492481484487.8 / in scientifc notation: 1.63049e+15


In [25]:
## 2. features with weight zero
features_zw(simple_features, weights)

[('bedrooms', 0.0)]

# Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets.

In [26]:
train_data, test_data = sales.random_split(.8, seed=0)

Let us consider the following set of features.

In [27]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

In [28]:
def run_lasso(initial_weights, l1_penalty, tolerance):
  return lasso_cyclical_coordinate_descent(
    norm_tr_feature_matrix, tr_output, 
    initial_weights, 
    l1_penalty, 
    tolerance
  )

First, create a normalized feature matrix from the *training* data with these features.  <br />
(*Make sure to store the norms for the normalization, since we'll use them later*)

In [29]:
my_ouput = 'price'
(tr_feature_matrix, tr_output) = get_numpy_data(train_data, all_features, my_output)
(norm_tr_feature_matrix, norms_tr) = normalize_features(tr_feature_matrix)

norm_tr_feature_matrix.shape, norms_tr.shape

((17384, 14), (14,))

First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e7`, you will need them later.

In [30]:
l1_penalty=1e7
initial_weights = np.zeros(len(all_features) + 1)
tolerance = 1.0

weights1e7 = run_lasso(initial_weights, l1_penalty, tolerance)
weights1e7.shape

(14,)

***Quiz Question***

What features had non-zero weight in this case?

In [31]:
features_nzw(all_features, weights1e7)

[('intercept', 24429600.609333135),
 ('sqft_living', 48389174.35227978),
 ('waterfront', 3317511.162719814),
 ('view', 7329961.984896399)]

Next, learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e8`, you will need them later.

In [32]:
l1_penalty=1e8
initial_weights = np.zeros(len(all_features) + 1)
tolerance = 1.0

weights1e8 = run_lasso(initial_weights, l1_penalty, tolerance)

***Quiz Question***

What features had non-zero weight in this case?

In [33]:
features_nzw(all_features, weights1e8)

[('intercept', 71114625.75280938)]

Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=5e5`.  <br />
Call resulting weights `weights1e4`, you will need them later.  <br />

**(This case will take quite a bit longer to converge than the others above.)**

In [34]:
l1_penalty=1e4
initial_weights = np.zeros(len(all_features) + 1)
tolerance = 1.0

weights1e4 = run_lasso(initial_weights, l1_penalty, tolerance)

***Quiz Question***

What features had non-zero weight in this case?

In [35]:
features_nzw(all_features, weights1e4)

[('intercept', 798641911.8458087),
 ('bathrooms', 13120484.354738437),
 ('sqft_living', 52120627.66093406),
 ('floors', 3881920.197285044),
 ('waterfront', 6863647.937280521),
 ('view', 4532741.1265115645),
 ('condition', 8612159.114729196),
 ('grade', 128519189.21940285),
 ('sqft_basement', 227011.96094560373),
 ('yr_renovated', 632131.4744055572)]

## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

In [36]:
all_weights = [
  w / norms_tr for w in  (weights1e4, weights1e7, weights1e8)
]

In [37]:
exp =  161.31745624837794
assert abs(all_weights[1][3] - exp) <= EPS

To check your results, if you call `normalized_weights1e7` the normalized version of `weights1e7`, then:
```
print normalized_weights1e7[3]
```
should return 161.31745624837794.

## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [38]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [39]:
best_rss, best_model = None, None

for (weights, label) in zip(all_weights, ('weights1e4', 'weights1e7', 'weights1e8')):
  rss = calc_rss(test_feature_matrix, test_output, weights)
  print(f"for {label} => rss: {rss} / in scientifc notation: {rss:1.5e}")
  if best_rss is None or rss < best_rss:
    best_rss, best_model = rss, label

for weights1e4 => rss: 194495188687674.0 / in scientifc notation: 1.94495e+14
for weights1e7 => rss: 275962079909185.28 / in scientifc notation: 2.75962e+14
for weights1e8 => rss: 537166150034084.9 / in scientifc notation: 5.37166e+14


***Quiz Question***

Which model performed best on the test data?

In [40]:
best_model

'weights1e4'